In [24]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, MinuteLocator, date2num
import datetime
import ipywidgets as widgets
import numpy as np
from IPython.display import HTML

file = "Data//831C_11471-20211117_Raw.csv" # location of processed CSV file (pre-processing speeds up read time)
NPS1raw = pd.read_csv(file,index_col="Date", parse_dates=True) # reading processed csv with index as 'Date' column
NPS1ThirdCols = NPS1raw.loc[:, NPS1raw.columns.str.startswith('1/3')].columns # list of 3rd octave columns
#start = "2021-11-17 16:50:00" # Set start time / date required
#start_dt = datetime.datetime.strptime(start, '%Y-%m-%d %H:%M:%S') # convert start time from text to datetime format
start_dt = NPS1raw.index[0]

# formatting matplotlib to create spectrogram 
%matplotlib widget
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, figsize=(10,7), gridspec_kw={'height_ratios': [1, 6, 4]}, sharex=True, tight_layout=True)

Cols = [col.replace("1/3 LZeq ","")+"Hz" for col in NPS1ThirdCols] # Formatting columns
tickloc = [x + 0.5 for x in range(0, len(Cols))] # moving y tick location to center of pixel line

def prevf(b):
    deltain = pd.Timedelta(hours=-1)
    update(deltain, Prec.value, Wind.value)

def nextf(b):
    deltain = pd.Timedelta(hours=1)
    update(deltain, Prec.value, Wind.value)

def changef(change):
    update(pd.Timedelta(hours=0), Prec.value, Wind.value)
        
def update(delta, p, w):
    global start_dt, NPS1raw
    NPS1rawplot = NPS1raw   
    start_dt += delta
    end_dt = start_dt+pd.Timedelta(hours=1) # add one hour to start time for end time
    NPS1rawplot = NPS1rawplot[start_dt:end_dt] # filtering by start time for end time    
    if w:
        NPS1rawplot = NPS1rawplot[NPS1rawplot["Wind Speed (m/s)"]<=5] # Removing wind speed above 5m/s      
    dtrange = pd.date_range(start=start_dt, end=end_dt, freq='S')
    NPS1rawplot = NPS1rawplot.reindex(dtrange) # reindexing to maintain time chart time period        
    NPS1thirdsplot = NPS1rawplot.loc[:, NPS1ThirdCols] # filter data with only third-octave columns
    startfloat, endfloat = date2num(start_dt), date2num(end_dt) # Converts datetime because imshow only accepts float values
    
    plt.suptitle("1 HOUR PERIOD STARTING "+str(start_dt), fontsize=10,x=0.8,y=0.88) # Title

    #plot 1: Table
    NPSLAeqvals = NPS1rawplot['LAeq'].dropna() # drop nan values before calculation
    LAeq = 10*np.log10(np.average(np.power(10,np.divide(NPSLAeqvals,10)))) # LAeq calc
    df = {'Date / Time':[start_dt],'L$_{eq}$ (dBA)':LAeq} # table formatting
    table1 = pd.DataFrame(df).round(1) 
    ax1.clear() # clears the plot (axis level)
    ax1.table(cellText=table1.values, colLabels=table1.columns, loc='center',cellLoc='left',colLoc='left',colColours =["lightgrey"]*2,bbox=[0,0,0.5,1.1])
    ax1.axis('off')

    #plot 2: Spectrogram    
    ax2.clear() # clears the plot (axis level)
    ax2.imshow(NPS1thirdsplot.T, aspect="auto", origin='lower', extent=[startfloat,endfloat,0,len(Cols)], interpolation=None) # plot spectrogram
    ax2.yaxis.set(ticks=tickloc) # setting tick locaiton
    ax2.set_yticklabels(Cols, fontsize=6) # setting y tick labels
    ax2.set_xlabel('Time') # x label
    ax2.set_ylabel('1/3 Octave band center frequency') # y label

    #plot 3: Time History  
    ax3.clear() # clears the plot (axis level)
    ax3.scatter(NPS1rawplot.index,NPS1rawplot['LAeq'],c='black',marker='.',s=3)
    ax3.set_xlabel('Time') # x label
    ax3.set_ylabel('L$_{EQ}$ dBA') # y label
    myFmt = DateFormatter('%H:%M') # formatter for x axis datetime (##:##)
    minutes = MinuteLocator(interval = 5) # setting x axis formatting / interval
    ax3.xaxis.set_major_locator(minutes) 
    ax3.xaxis.set_major_formatter(myFmt)
    fig.autofmt_xdate()
    fig.show()

box_layout = widgets.Layout(display='inline-flex',flex_flow='row',align_items='stretch',margin='0px 0px 10px 140px')
prev = widgets.Button(description="Previous hour")
next = widgets.Button(description="Next hour")
Prec = widgets.Checkbox(value=False,description='Filter precipitation',disabled=False)
Wind = widgets.Checkbox(value=False,description='Filter wind speeds above 11mph',disabled=False)
ui = widgets.HBox([prev,next,widgets.VBox([Prec,Wind])],layout=box_layout)
out = widgets.Output()
display(ui, out)

update(pd.Timedelta(hours=0), Prec.value, Wind.value)
prev.on_click(prevf)
next.on_click(nextf)
Prec.observe(changef,names='value')
Wind.observe(changef,names='value')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()